In [5]:
import gradio as gr
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.autograd import Variable
from PIL import Image
import json
import torch.optim as optim
from torch.optim import lr_scheduler



def image_loader(image_name):
    image = loader(image_name).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image  #assumes that you're using GPU




#load model
vgg16 = models.vgg16_bn()



    # Inable our saved parameters
for param in vgg16.features.parameters():
    param.require_grad = False
    
    # Newly created modules have require_grad=True by default
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1]  # Remove last layer
features.extend([nn.Linear(num_features, 3)])  # Add our layer with 3 outputs
vgg16.classifier = nn.Sequential(*features)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
#load our saved model
vgg16 = vgg16.load_state_dict(torch.load('/Users/matthew/PycharmProjects/CSCI338_Project/VGG16_graphs.pt',map_location='cpu'))

vgg16.eval()

#freeze parameters so we don't overtrain it


loader = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

f = open("/Users/matthew/PycharmProjects/CSCI338_Project/outsideCode/classes.txt","r")
labeled_classes = json.loads(f.read())
labels = [i.split(',')[0] for i in list(labeled_classes.values())]

gr.Interface(fn=predict,
             inputs=gr.inputs.Image(type="pil"),
             outputs=gr.outputs.Label(num_top_classes=3)).launch()

AttributeError: '_IncompatibleKeys' object has no attribute 'eval'